In [6]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import itertools
from sklearn.model_selection import cross_val_score, KFold
from sklearn.feature_selection import VarianceThreshold

In [7]:
kf = KFold(n_splits = 10, shuffle = True, random_state= 42)

In [8]:
df = pd.read_csv('logP.csv')
y = df.logP
X = df.iloc[:, 3:]
feats = X.columns
X

,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.909,12.053455,12.053455,0.045556,0.045556,0.909253,281.355,262.203,281.141579,108,...,0,0,0,0,0,0,0,0,0,0
1,0.839,12.616874,12.616874,0.009514,-5.052156,0.839542,262.183,253.111,262.045293,98,...,0,0,0,0,0,0,0,0,0,0
2,0.828,4.217020,4.217020,1.056122,1.056122,0.828333,266.388,244.212,266.178299,104,...,0,0,0,0,0,0,0,0,0,0
3,0.726,11.239605,11.239605,0.364327,-1.508792,0.726080,318.372,300.228,318.125594,120,...,0,0,0,0,0,0,0,0,0,0
4,0.816,4.352931,4.352931,0.140811,0.140811,0.816043,214.312,196.168,214.146999,84,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.863,12.562081,12.562081,0.093167,-0.455663,0.863227,381.501,358.317,381.151098,140,...,1,0,0,0,0,0,0,0,0,0
240,0.737,12.114099,12.114099,0.072506,-0.211498,0.737878,321.376,302.224,321.136493,122,...,0,0,0,0,0,0,0,0,0,0
241,0.824,12.974979,12.974979,0.339313,-1.732014,0.824192,274.266,262.170,274.080536,102,...,0,0,0,0,0,0,0,0,0,0
242,0.750,12.347197,12.347197,0.125336,-0.724707,0.750035,321.312,308.208,321.088080,120,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X.describe()

,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.00000,244.000000,244.000000,...,244.000000,244.000000,244.000000,244.0,244.000000,244.000000,244.0,244.000000,244.000000,244.000000
mean,0.786852,11.785110,11.785110,0.148314,-1.340510,0.786676,331.484148,312.88159,331.075276,121.114754,...,0.110656,0.127049,0.024590,0.0,0.004098,0.045082,0.0,0.110656,0.024590,0.045082
std,0.068484,2.102695,2.102695,0.156852,1.734924,0.069262,61.981447,59.61362,61.863649,22.510792,...,0.327180,0.357526,0.155191,0.0,0.064018,0.226842,0.0,0.339525,0.201358,0.244311
min,0.672000,4.217020,4.217020,0.001181,-5.843145,0.623848,184.242000,172.14600,184.100048,66.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,0.727750,11.866152,11.866152,0.044773,-3.140529,0.727401,295.087750,274.21400,294.840215,105.500000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,0.782000,12.409469,12.409469,0.101036,-0.483724,0.782126,334.442500,316.93700,334.124549,122.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
75%,0.840250,12.868995,12.868995,0.201364,-0.158534,0.840644,369.374000,350.19925,369.124046,136.500000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
max,0.942000,14.730765,14.730765,1.056122,1.056122,0.942487,483.322000,464.17000,482.058967,170.000000,...,2.000000,2.000000,1.000000,0.0,1.000000,2.000000,0.0,2.000000,2.000000,2.000000


In [10]:
X.QED_score - X.QED_score. / X.QED_score.std()

SyntaxError: invalid syntax (270965344.py, line 1)

In [ ]:
X.MolWt / X.MolWt.std()

0      4.539342
1      4.230024
2      4.297867
3      5.136569
4      3.457680
         ...   
239    6.155084
240    5.185035
241    4.424969
242    5.184003
243    6.623627
Name: MolWt, Length: 244, dtype: float64

In [11]:
selector = VarianceThreshold(10)
selected_features = selector.fit_transform(X, y)
selected_features_index = selector.get_support(indices=True)
selected_feature_names = X.columns[selected_features_index]
X_new = pd.DataFrame(selected_features, columns = selected_feature_names)
X_new


,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,BCUT2D_MWHI,BertzCT,Ipc,LabuteASA,PEOE_VSA1,PEOE_VSA10,...,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState6,VSA_EState7,HeavyAtomCount,MolMR
0,281.355,262.203,281.141579,108.0,16.465851,706.359669,6.975974e+04,124.497248,10.053652,5.749512,...,31.296997,4.736863,5.471081,0.000000,12.053455,2.985394,12.173224,0.454240,21.0,83.9997
1,262.183,253.111,262.045293,98.0,19.413883,497.551959,7.977097e+03,99.929735,14.580253,11.499024,...,4.736863,4.736863,47.192212,0.000000,11.566614,9.379690,3.854776,-6.209072,18.0,53.5743
2,266.388,244.212,266.178299,104.0,15.059973,524.320217,3.216838e+04,121.508214,9.799819,0.000000,...,64.909692,0.000000,0.000000,0.000000,4.191308,0.000000,16.969155,0.000000,20.0,89.6740
3,318.372,300.228,318.125594,120.0,16.364915,814.604445,4.328251e+05,140.752324,14.630206,11.520495,...,11.840869,4.417151,5.387791,0.000000,0.000000,21.452123,21.820029,-1.099399,24.0,92.0636
4,214.312,196.168,214.146999,84.0,15.120847,449.087623,5.439498e+03,96.710470,0.000000,0.000000,...,61.300411,0.000000,0.000000,0.000000,0.000000,7.468883,12.581179,0.896250,16.0,66.5357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,381.501,358.317,381.151098,140.0,32.166721,903.142781,1.118114e+06,163.418318,5.316789,5.249938,...,5.316789,4.992405,0.000000,1.346699,31.169713,3.039571,13.598590,0.118607,27.0,111.5857
240,321.376,302.224,321.136493,122.0,16.521215,844.916502,3.832465e+05,140.568405,14.790515,0.000000,...,5.316789,9.473726,11.053188,0.000000,12.114099,5.068889,21.145723,0.000000,24.0,95.6837
241,274.266,262.170,274.080536,102.0,19.143514,591.417357,3.332934e+04,114.867029,10.213055,18.241324,...,11.840869,5.106527,25.949958,0.000000,0.000000,19.544116,10.369830,0.000000,20.0,70.6606
242,321.312,308.208,321.088080,120.0,16.471022,988.066548,2.911067e+05,136.632268,14.840467,0.000000,...,10.309193,4.417151,5.130607,0.000000,27.445004,14.916513,13.230576,1.202396,24.0,89.9297


In [12]:
feats2 = X_new.columns
feats2

Index(['MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons',
       'BCUT2D_MWHI', 'BertzCT', 'Ipc', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10',
       'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2',
       'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7',
       'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA3',
       'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA9',
       'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA12', 'SlogP_VSA2',
       'SlogP_VSA3', 'SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA8',
       'TPSA', 'EState_VSA1', 'EState_VSA10', 'EState_VSA2', 'EState_VSA3',
       'EState_VSA4', 'EState_VSA5', 'EState_VSA6', 'EState_VSA7',
       'EState_VSA8', 'EState_VSA9', 'VSA_EState1', 'VSA_EState10',
       'VSA_EState2', 'VSA_EState3', 'VSA_EState6', 'VSA_EState7',
       'HeavyAtomCount', 'MolMR'],
      dtype='object')

In [30]:
def feature_selection_cv(X, y, features: list, n_features = 2):

    r2_values = []

    for n_feat in tqdm(range(1, n_features+1)):

        combs = itertools.combinations(features, n_feat)

        for feature_subset in combs:

            model = linear_model.LinearRegression()

            X_train = X[[*feature_subset]]

            coeff = cross_val_score(model, X_train , y, cv = kf).mean()
            r2_values.append((n_feat, coeff, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['n', 'r2', 'cols'])

    return r2_df.groupby('n').max()


In [32]:
res = feature_selection_cv(X_new, y, feats2, 3)
res

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [06:15<00:00, 125.08s/it]


,r2,cols
n,,
1,0.257349,VSA_EState7
2,0.328974,VSA_EState7 MolMR
3,0.411825,VSA_EState7 HeavyAtomCount MolMR


In [22]:
model2222 = linear_model.LinearRegression()
score = cross_val_score(model2222, X_new , y, cv = kf).mean()
score

0.3466392350820645